In [8]:
#import pandas as pd
import json
import numpy as np
from tqdm import tqdm_notebook as tqdm
import uuid
import weaviate 
from weaviate.batch import Batch 
import helper
import weaviate

# initiate the Weaviate client
client = weaviate.Client("http://localhost:8081")
client.timeout_config = (3, 200)


In [11]:
# load all metadata
productData = []
with open('data/sample_meta_Home_and_Kitchen.json', 'r') as f:
    for l in tqdm(f):
        productData.append(json.loads(l))


/var/folders/4c/x3415rcx4pb9drxwpfyzk32h0000gn/T/ipykernel_72086/3156423274.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for l in tqdm(f):


0it [00:00, ?it/s]

In [2]:
client.schema.delete_all() # delete all classes
import weaviate

# skipped product "brand", it can be it's own class, for simplicity, skip it for now 
# skipped product "category" list of strings (text?) not sure how to type it yet, or if category should be it's own class?
# skipped "similar_item"

schema = {
    "classes": [
     {
            # name of the class
            "class": "Product",
            # a description of what this class represents
            "description": "A Product class to store the product information and its reviewers",
            # class properties
            "properties": [
                {
                    "name": "asin",
                    "dataType": ["string"],
                    "description": "product id from amazon", 
                },
                {
                    "name": "title",
                    "dataType": ["text"],
                    "description": "The title of the product", 
                },{
                    "dataType": ["number"],
                    "description": "The price product in dollars",
                    "name": "price"
                },{
                    "name": "description",
                    "dataType": ["text"],
                    "description": "The first item in description  of the product", 
                }
                
            ]
        }
            ]
}

client.schema.create(schema)

In [15]:
def prettify(json_dict): 
    print(json.dumps(json_dict, indent=2))

#prettify(client.schema.get())

In [12]:
helper.generate_uuid('product', productData[350]['title']+productData[350]['asin'])

'fe664d69-5fa5-52ef-b2c9-0014f95f30e5'

In [18]:
productData[350]['description'][0]

'ty: 5 bags and filters Bosch G bag BBZ51AFG1U Type G Three layers for better dust retention. 99.97% of all particles remain in the dust bag. Replaces quickly, cleanly and easily. As you pull the tab to remove the bag it automatically closes and seals in the dirt. 1 gallon / 3.5 liter capacity.BBZ51AFG1U Type G Fits BSA2... series canister vacuum cleaner'

In [ ]:
from weaviate.batch import Batch # for the typing purposes


def add_product(batch: Batch, product_data: dict) -> str:
    product_object = {
        'asin': product_data['asin'] if 'asin' in product_data else '',
        'title': product_data['title'] if 'title' in product_data else '',
        #'main_cat': product_data['main_cat'] if 'main_cat' in product_data else '',
        'description':product_data['description'][0] if 'description' in product_data else '',               
        #'feature': product_data['feature'] if 'feature' in product_data else '',
        'price': float(product_data['price'].replace("$","").replace(",","")) if 'price' in product_data else float("NaN")
    }
    # generate an UUID for the Author
    product_id = generate_uuid('product', product_data['title']+product_data['asin'])
   
    # add article to the batch
    batch.add_data_object(  # old way was batch.add(...)
        data_object=product_object,
        class_name='Product',
        uuid=product_id
    )
    
    return product_id
